# Creación panel de coca

In [1]:
import os
import itertools
import numpy as np
import time
import re
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from scipy.spatial import cKDTree
from shapely import Point, LineString, get_coordinates
import pyarrow

In [2]:
user=os.environ.get('USER')

#Rutas:
if user=="santiagoherreragarcia":
    root="/Users/santiagoherreragarcia/Library/CloudStorage/OneDrive-Bibliotecascompartidas:Universidaddelosandes/Carolina Castro Osorio - REDD+ bases"
    densidades="/Users/santiagoherreragarcia/Library/CloudStorage/OneDrive-Bibliotecascompartidas:Universidaddelosandes/Carolina Castro Osorio - REDD+ bases/Inputs/Coca/Centroides"
elif user=="carolinacastroosorio":
    root="/Users/carolinacastroosorio/Library/CloudStorage/OneDrive-UniversidaddelosAndes/REDD+ bases"
    densidades="/Users/carolinacastroosorio/Library/CloudStorage/OneDrive-UniversidaddelosAndes/REDD+ bases/Inputs/Coca/Centroides"
elif user=="USER":
    root="C:/Users/USER/OneDrive - Universidad de los Andes (1)/REDD+ bases"
    densidades="C:/Users/USER/OneDrive - Universidad de los Andes (1)/REDD+ bases/Inputs/Coca/Centroides"
elif user=="ccastro":
    root="C:/Users/ccastro/OneDrive - Universidad de los andes/REDD+ bases"
    densidades="C:/Users/ccastro/OneDrive - Universidad de los andes/REDD+ bases/Inputs/Coca/Centroides"

Agregar polígonos de coca

In [4]:
# Temporal
user=os.environ.get('USER')

if user=="ccastro":
    root="C:/Users/ccastro/OneDrive - Universidad de los andes/REDD+ bases"
    densidades="C:/Users/ccastro/OneDrive - Universidad de los andes/REDD+ bases/Inputs/Coca/Centroides"


In [6]:
#Cuadrícula completa para el Pacífico de 1km2

#os.chdir(root+"\Inputs")
#grid_pacifico=gpd.read_file('C:/Users/ccastro/OneDrive - Universidad de los andes/REDD+ bases/Inputs/Grid Pacifico')
#C:\Users\ccastro\OneDrive - Universidad de los andes\REDD+ bases\Inputs

#Arreglar Grid
grid_pacifico['id']=range(0,len(grid_pacifico))
grid_pacifico=grid_pacifico.loc[:,['id', 'geometry']]
grid_pacifico['centroids']=grid_pacifico['geometry'].centroid


FileNotFoundError: [Errno 2] No such file or directory: '/Users/carolinacastroosorio/Library/CloudStorage/OneDrive-UniversidaddelosAndes/REDD+ bases\\Inputs'

Explicación de lo que viene: for i in os.listdir():: Inicia un bucle for que recorre los archivos en el directorio actual (el cual se cambió previamente usando os.chdir(densidades)).

if '.DS' not in i:: Verifica si el nombre del archivo (i) no contiene '.DS' en su nombre. Esto es probablemente para evitar archivos ocultos generados en sistemas macOS que podrían causar problemas al intentar leerlos.

df=gpd.read_file(i): Lee el archivo geoespacial (shapefile u otro formato compatible) especificado por el nombre de archivo i utilizando GeoPandas y almacena los datos en el DataFrame df.

centroides_densidades.append(df): Agrega el DataFrame df (que contiene los datos de un archivo de centroides) a la lista centroides_densidades. Esto acumula todos los DataFrames leídos en la lista.

centroides_densidades=rdf = gpd.GeoDataFrame(pd.concat(centroides_densidades, ignore_index=True), crs=centroides_densidades[0].crs): Concatena todos los DataFrames almacenados en la lista centroides_densidades en un único GeoDataFrame utilizando pd.concat(). El argumento ignore_index=True asegura que los índices se vuelvan a generar. El GeoDataFrame resultante se asigna a la variable centroides_densidades, y también se asigna a rdf (aunque esto último no parece necesario).

centroides_densidades.sort_values(by='ano', inplace=True): Ordena el GeoDataFrame centroides_densidades según los valores en la columna 'ano' en su lugar, utilizando el método sort_values(). El argumento inplace=True hace que la operación de clasificación se realice directamente en el GeoDataFrame sin necesidad de asignación adicional.

In [28]:
#Leer archivos de centroides para concatenar
densidades='C:/Users/ccastro/OneDrive - Universidad de los andes/REDD+ bases/Inputs/Coca/Centroides'
#Se crea una lista vacía llamada centroides_densidades
centroides_densidades=[]
os.chdir(densidades)


for a in centroides_densidades.ano.drop_duplicates():
    dens=centroides_densidades.loc[centroides_densidades.ano==a,['geometry','ano','areacoca']]
    tr_a=grid_pacifico.sjoin(dens, how='left', predicate='intersects')
    tr_a['ano']=a
    tr_a=tr_a.drop(columns='index_right').drop_duplicates()
    trans.append(tr_a)
panel_grid=pd.concat(trans)
panel_grid.sort_values(by=['id', 'ano'], inplace=True)
panel_grid.reset_index(drop=True, inplace=True)
panel_grid.fillna(0, inplace=True)
    if '.DS' not in i: 
        df=gpd.read_file(i)
        centroides_densidades.append(df)
centroides_densidades=rdf = gpd.GeoDataFrame(pd.concat(centroides_densidades,
                                                ignore_index=True), crs=centroides_densidades[0].crs)
centroides_densidades.sort_values(by='ano', inplace=True)

Comentario: 
trans=[]: Se inicia una lista vacía llamada trans que se utilizará para almacenar los resultados de las transformaciones en el bucle.

for a in centroides_densidades.ano.drop_duplicates():: Inicia un bucle for que itera a través de los años únicos presentes en la columna 'ano' del DataFrame centroides_densidades. Esto significa que el bucle se ejecutará una vez para cada año presente en los datos.

dens=centroides_densidades.loc[centroides_densidades.ano==a,['geometry','ano','areacoca']]: Crea un DataFrame dens filtrando el DataFrame centroides_densidades para obtener las filas correspondientes al año actual (a). Solo se seleccionan las columnas 'geometry', 'ano' y 'areacoca'.

tr_a=grid_pacifico.sjoin(dens, how='left', predicate='intersects'): Realiza una unión espacial izquierda entre el DataFrame dens y el DataFrame grid_pacifico, utilizando la columna 'geometry' para determinar la intersección espacial. El resultado se almacena en el DataFrame tr_a.

tr_a['ano']=a: Agrega una columna 'ano' al DataFrame tr_a y le asigna el valor del año actual (a).

tr_a=tr_a.drop(columns='index_right').drop_duplicates(): Elimina la columna 'index_right' (posiblemente generada por la unión espacial) del DataFrame tr_a y elimina filas duplicadas basadas en todas las columnas.

trans.append(tr_a): Agrega el DataFrame tr_a a la lista trans, que contiene los resultados de todas las transformaciones.

panel_grid=pd.concat(trans): Concatena todos los DataFrames en la lista trans en un solo DataFrame llamado panel_grid.

panel_grid.sort_values(by=['id', 'ano'], inplace=True): Ordena el DataFrame panel_grid primero por la columna 'id' y luego por la columna 'ano' en su lugar, utilizando el método sort_values().

panel_grid.reset_index(drop=True, inplace=True): Restablece el índice del DataFrame panel_grid y elimina la columna del índice anterior.

panel_grid.fillna(0, inplace=True): Rellena los valores faltantes (NaN) en el DataFrame panel_grid con ceros.

In [29]:
#Crear panel anual

trans=[]
for a in centroides_densidades.ano.drop_duplicates():
    dens=centroides_densidades.loc[centroides_densidades.ano==a,['geometry','ano','areacoca']]
    tr_a=grid_pacifico.sjoin(dens, how='left', predicate='intersects')
    tr_a['ano']=a
    tr_a=tr_a.drop(columns='index_right').drop_duplicates()
    trans.append(tr_a)
panel_grid=pd.concat(trans)
panel_grid.sort_values(by=['id', 'ano'], inplace=True)
panel_grid.reset_index(drop=True, inplace=True)
panel_grid.fillna(0, inplace=True)

Agregar Consejos Comunitarios para identificar polígonos tratados y no tratados

In [35]:
#cc=gpd.read_file(root+'/Inputs/Consejos Comunitarios', encoding='utf-8')
cc=gpd.read_file(r"C:/Users/ccastro/OneDrive - Universidad de los andes/REDD+ bases/Inputs/Consejos Comunitarios")
# Selección de Columnas Útiles y Eliminación de Duplicados
cc=cc.loc[:,['NOMBRE_COM', 'ANO', 'RESOLUCION', 'geometry']].drop_duplicates()

# Agregamos los Consejos Comunitarios a grid_pacifico. Se hace la intersección y se eliminan lineas duplicadas
grid_pacifico=grid_pacifico.set_geometry('centroids')
grid_pacifico=grid_pacifico.sjoin(cc, how='left', predicate='intersects')
grid_pacifico=grid_pacifico.drop(columns='index_right').drop_duplicates(subset=['id'],keep='first')

In [36]:
# Lista de Consejos Comunitarios tratados

tratados=["Bajo Mira y Frontera", "Acapa", "Cajambre", "La Plata", "Calima", "Madre",
"Concosta", "Río Pepe", "Chicao", "Buenavista", "Domingodó", "Montaño", "Sivi", "Usaragá", "Pizarro",
"Piliz", "Acaba", "Tolo", "Vígia De Curvaradó Y Santa Rosa De Limón", "Apartadó Buenavista", 
'General De La Costa Pacífica Del Norte - Los Delfines','Cupica','Mayor Del Cantón De San Pablo "Acisanp"',  
'Consejo Comunitario Integral De Lloro_Cocoillo','Mayor De Novita', 'Pedeguita Y Mancilla',  'Cértegui',  
'Río Jiguamiandó',  'La Cuenca Del Río Tolo Y Zona Costera Sur',  'San Andrés De Usaragá',  'Sivirù',  
'Pizarro', 'Río  Pilizá', 'General Del Municipìo De Nuquí - Los Riscales',  'Río Domingodó',  
'Consejo Comunitario De La Madre',  'Consejo Comunitario De Chicao',  'Río Curvaradó',  'Río Montaño',  
'Apartadó Buenavista',  'Río Pepe', 'Río Baudó  Acaba',  'Mayor Del Municipio De Condoto',  'Acapa',  'Mayor Del Medio Atrato - Acia',  
'Bajo Mira Y Frontera',"Alto Guapi","Mayor Del Municipio De Juradó", "Mayor De Novita", "Rio Naya", "Cuenca Baja Del Río Calima", "Bahía Málaga -La Plata","El Río Cajambre" ]

# Dummy por cada año de tratamiento
cohorte_2009=["La Cuenca Del Río Tolo Y Zona Costera Sur"]
cohorte_2011=["General De La Costa Pacífica Del Norte - Los Delfines"]
cohorte_2014=["Cuenca Baja Del Río Calima", "Bajo Mira Y Frontera", "Acapa","Consejo Comunitario De Chicao","Consejo Comunitario De La Madre",
               "Apartadó Buenavista", "Río Domingodó", "Vígia De Curvaradó Y Santa Rosa De Limón", "Río Montaño",
               "Río Pepe", "Río Baudó  Acaba","Bahía Málaga -La Plata", "El Río Cajambre", "Concosta", "Sivirú","San Andrés De Usaragá", "Pizarro", "Río  Pilizá" ]
cohorte_2015=['Mayor Del Cantón De San Pablo "Acisanp"', 'Consejo Comunitario Integral De Lloro_Cocoillo', "Cértegui", 'General Del Municipìo De Nuquí - Los Riscales']
cohorte_2016=['Rio Naya', "Mator De Novita", 'Mayor Del Municipio De Juradó']
cohorte_2017=['Mayor Del Medio Atrato - Acia']
cohorte_2018=['Alto Guapi']
cohorte_2019=["Mayor Del Municipio De Condoto"]
cohorte_2020=['Pedeguita Y Mancilla','Río Jiguamiandó']


In [37]:
# Asignar tratados y no tratados

grid_pacifico['Di']=np.where(grid_pacifico.loc[:,'NOMBRE_COM'].isin(tratados),1,0)

#Creo cada dummy 
grid_pacifico['2009']=np.where(grid_pacifico.loc[:,'NOMBRE_COM'].isin(cohorte_2009),1,0)
grid_pacifico['2011']=np.where(grid_pacifico.loc[:,'NOMBRE_COM'].isin(cohorte_2011),1,0)
grid_pacifico['2014']=np.where(grid_pacifico.loc[:,'NOMBRE_COM'].isin(cohorte_2014),1,0)
grid_pacifico['2015']=np.where(grid_pacifico.loc[:,'NOMBRE_COM'].isin(cohorte_2015),1,0)
grid_pacifico['2016']=np.where(grid_pacifico.loc[:,'NOMBRE_COM'].isin(cohorte_2016),1,0)
grid_pacifico['2017']=np.where(grid_pacifico.loc[:,'NOMBRE_COM'].isin(cohorte_2017),1,0)
grid_pacifico['2018']=np.where(grid_pacifico.loc[:,'NOMBRE_COM'].isin(cohorte_2018),1,0)
grid_pacifico['2019']=np.where(grid_pacifico.loc[:,'NOMBRE_COM'].isin(cohorte_2019),1,0)
grid_pacifico['2020']=np.where(grid_pacifico.loc[:,'NOMBRE_COM'].isin(cohorte_2020),1,0)


El último paso es pegar las dos bases de datos y exportar el resultado.

In [38]:
base_coca=panel_grid.merge(grid_pacifico)

In [39]:
len(base_coca)==len(panel_grid) #Todas las observaciones deben hacer match.

True

In [40]:
#Exportación
#os.chdir(root+"/Outputs Bases")
os.chdir(r"C:/Users/ccastro/OneDrive - Universidad de los andes/REDD+ bases/Outputs Bases")
base_coca.to_parquet("base_coca.parquet")

In [7]:
print(base_coca.head())

NameError: name 'base_coca' is not defined

In [42]:
tend_plt=sns.lineplot(data=base_coca, x='ano', y='mean_coca', hue='Di')

NameError: name 'sns' is not defined

In [6]:
import matplotlib.pyplot as plt

# Mapea los valores de 'Di' a las etiquetas deseadas
di_labels = {1: 'Tratados', 0: 'Controles'}
base_coca['Di_Label'] = base_coca['Di'].map(di_labels)

plt.figure(figsize=(10, 6))  # Tamaño del gráfico (ancho, alto)

# Itera sobre cada grupo 'Di_Label' y grafica el promedio anual de 'areacoca'
for category, group in base_coca.groupby('Di_Label'):
    annual_mean_areacoca = group.groupby('ano')['areacoca'].mean()
    plt.plot(annual_mean_areacoca.index, annual_mean_areacoca.values, marker='o', label=category)

plt.xlabel('Año')  # Etiqueta para el eje x
plt.ylabel('Promedio Anual de Área de Coca')  # Etiqueta para el eje y
plt.title('Promedio Anual de Área de Coca por Grupo Di')  # Título del gráfico
plt.legend()  # Muestra leyendas para cada categoría 'Di_Label'
plt.grid(True)  # Agrega una cuadrícula al gráfico
plt.show()  # Muestra el gráfico


NameError: name 'base_coca' is not defined

## NASA FDLAS

In [2]:
fldas=gpd.read_parquet('/Users/carolinacastroosorio/Library/CloudStorage/OneDrive-UniversidaddelosAndes/REDD+ bases/Inputs/FLDAS_2001_2020.parquet')
fldas['ano']=fldas.quarter.apply(lambda x: re.search(r'\d+',x).group(0)).astype(int)
fldas['quarter']=fldas.quarter.apply(lambda x: re.search(r'Q\d',x).group(0))
fldas.drop(columns='coords', inplace=True)
fldas=fldas.pivot(index=['geometry', 'ano'], columns=['quarter'])
fldas.columns = fldas.columns.to_series().str.join('_')
fldas.reset_index(inplace=True)
fldas=gpd.GeoDataFrame(fldas, crs="EPSG:3116")

In [ ]:
panel_grid['centroids']=panel_grid.centroid
panel_grid=panel_grid.set_geometry('centroids')
grid_pacifico['centroids']=grid_pacifico.centroid
grid_pacifico=grid_pacifico.set_geometry('centroids')

In [ ]:
#Crear panel anual (NASA)
trans_nasa=[]
for a in fldas.ano.drop_duplicates():
    fldas_a=fldas.loc[fldas.ano==a,:]
    tr_a=grid_pacifico.sjoin(fldas_a, how='left', predicate='intersects')
    tr_a['ano']=a
    tr_a=tr_a.drop(columns='index_right').drop_duplicates()
    trans_nasa.append(tr_a)
nasa_grid=pd.concat(trans_nasa)
nasa_grid.sort_values(by=['id', 'ano'], inplace=True)
nasa_grid.reset_index(drop=True, inplace=True)

In [ ]:
def ckdnearest(gdfA, gdfB, geomcolA, geomcolB, namedist,anycol=False, colsB=None):
    
    original_idx=gdfB.index
    A = np.concatenate(
        [get_coordinates(geom) for geom in gdfA[geomcolA].to_list()])
   
    B = [get_coordinates(geom) for geom in gdfB[geomcolB].to_list()]
    
    B_ix = tuple(itertools.chain.from_iterable(
        [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]))

    B=np.concatenate(B)
    ckd_tree = cKDTree(B)
    dist, idx = ckd_tree.query(A, k=1)
    dist=pd.Series(dist, name=namedist)
    final_idx=[original_idx[B_ix[ix]] for ix in idx]
    
    if anycol==False:
        gdf= pd.concat([gdfA,dist], axis=1)
    else:
        gdf = pd.concat([gdfA.reset_index(drop=True),dist,gdfB.loc[final_idx,colsB].reset_index(drop=True)], axis=1)
    return gdf

def ckdnearest_imputer(df, cols_to_impute):
    
    """
    Imputación por distancia geográfica. La base original debe tener una columna llamada centroids para calcular
    las distancias
    """
    
    df=df.copy()
    #Partir la base en dos
    df1=df.loc[:,['centroids']+cols_to_impute]

    nas=df1.loc[df1.isnull().any(axis=1)]
    full=df1.loc[~df1.isnull().any(axis=1)]

    #Guardamos los indices
    nas_index=nas.index
    full_index=full.index

    #CDKTree
    A = np.concatenate(
        [get_coordinates(geom) for geom in nas['centroids'].to_list()])

    B = [get_coordinates(geom) for geom in full['centroids'].to_list()]

    B_ix = tuple(itertools.chain.from_iterable(
        [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]))

    B=np.concatenate(B)
    ckd_tree = cKDTree(B)
    dist, idx = ckd_tree.query(A, k=3) #Arbitrariamente son 3

    #Idx
    ind_list=[]
    final_index=[]
    for i in idx:
        ind_list.append([B_ix[k] for k in i])

    for l,e in enumerate(ind_list):
        final_index.append([full_index[B_ix[i]] for i in ind_list[l]])

    #Means
    for f,na in enumerate(nas_index):
        means=full.loc[final_index[f], cols_to_impute].mean(axis=0)
        df.loc[na, cols_to_impute]=means
        
    return df

In [ ]:
cols_to_impute=list(nasa_grid.columns[4:])

In [ ]:
nasa_grid_full=nasa_grid.groupby('ano').apply(lambda x: ckdnearest_imputer(x, cols_to_impute))

In [ ]:
nasa_grid_full.drop(columns=['geometry', 'centroids'], inplace=True)

In [ ]:
#Actualizar panel completo
panel_grid=panel_grid.merge(nasa_grid_full,how='left')

## Panel CEDE

In [51]:
cede=r'C:\Users\ccastro\OneDrive - Universidad de los andes\REDD+ bases\Inputs\Panel CEDE'

In [56]:
os.chdir(cede)
violencia=pd.read_excel('PANEL_CONFLICTO_Y_VIOLENCIA(2021).xlsx')

vars_interes=['terrorismot', #Acciones terroristas
              'secuestros', #Secuestros
              'acc_subversivas', #Total acciones subversivas
              'cont_arm', #Total de contactos armados con grupos ilegales
              'errad_manual', #Hectáreas erradicadas
              'homicidios'
              ]
violencia=violencia.loc[:,['codmpio', 'ano']+vars_interes]

OJOJOJOJOJOJO question xq reemplazar geometía original por centroides?

In [57]:
violencia

,codmpio,ano,terrorismot,secuestros,acc_subversivas,cont_arm,errad_manual,homicidios
0,5001.0,1993,NaN,NaN,NaN,NaN,NaN,NaN
1,5001.0,1994,NaN,NaN,NaN,NaN,NaN,NaN
2,5001.0,1995,NaN,NaN,NaN,NaN,NaN,NaN
3,5001.0,1996,NaN,81.0,NaN,NaN,NaN,NaN
4,5001.0,1997,NaN,74.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
32558,99001.0,2021,NaN,NaN,NaN,NaN,NaN,6.0
32559,99524.0,2021,NaN,NaN,NaN,NaN,NaN,1.0
32560,99624.0,2021,NaN,NaN,NaN,NaN,NaN,NaN
32561,99773.0,2021,NaN,NaN,NaN,NaN,NaN,4.0


Falta correr acá todo lo de FDLAS para poder seguir con lo de abajo

In [60]:
#Para pegar usamos el MGN

os.chdir(r'C:\Users\ccastro\OneDrive - Universidad de los andes\REDD+ bases\Inputs\MGN_2018')
mpios=gpd.read_file('MPIOS')
mpios=mpios.loc[:,['DPTO_CCDGO','MPIO_CCDGO','geometry']]
mpios['centroids']=mpios.centroid
mpios.set_geometry('centroids')

,DPTO_CCDGO,MPIO_CCDGO,geometry,centroids
0,18,001,"POLYGON ((850567.651 734450.429, 850602.776 73...",POINT (835222.404 685239.772)
1,18,029,"POLYGON ((797693.586 642855.101, 797693.823 64...",POINT (799100.771 627605.926)
2,18,094,"POLYGON ((809754.098 685339.982, 809743.624 68...",POINT (799867.206 657813.668)
3,18,247,"POLYGON ((857152.436 748523.984, 857180.000 74...",POINT (875773.988 689886.680)
4,18,256,"POLYGON ((856617.069 736147.468, 857365.135 73...",POINT (871299.804 670685.042)
...,...,...,...,...
1117,25,290,"POLYGON ((975638.881 980969.367, 975642.608 98...",POINT (965467.632 969855.245)
1118,25,662,"POLYGON ((927827.917 1041265.704, 927839.486 1...",POINT (935553.506 1027108.560)
1119,73,349,"POLYGON ((917504.922 1079725.053, 917524.682 1...",POINT (921778.141 1064536.303)
1120,05,631,"POLYGON ((831074.846 1172630.978, 831079.949 1...",POINT (830487.670 1170774.455)


In [61]:
grid_pacifico=ckdnearest(grid_pacifico, mpios, 'centroids', 'geometry', 'dist_mun', True,['DPTO_CCDGO','MPIO_CCDGO'])

NameError: name 'ckdnearest' is not defined

In [ ]:
grid_cc.drop(columns=['dist_mun'], inplace=True)

grid_cc['codmpio']=grid_cc['DPTO_CCDGO'].astype(str)+grid_cc['MPIO_CCDGO'].astype(str)
grid_cc['codmpio']=grid_cc['codmpio'].astype(int)

panel_cc=panel_cc.merge(grid_cc)

panel_cc.drop(columns=['DPTO_CCDGO', 'MPIO_CCDGO'], inplace=True)

panel_cc=panel_cc.merge(violencia, how='left')
panel_cc=panel_cc.fillna(0)

os.chdir(capas_3116)
#panel_cc.to_parquet('panel_cc.parquet')

grid_cc=grid_cc.drop(columns='centroids')
grid_cc=grid_cc.set_geometry('geometry')

#grid_cc.to_file('grid_cc')

panel_stat=ckdnearest(panel_grid, mpios, 'centroids', 'geometry', 'dist_mun', True,['DPTO_CCDGO','MPIO_CCDGO'])
panel_stat['codmpio']=panel_stat['DPTO_CCDGO'].astype(str)+panel_stat['MPIO_CCDGO'].astype(str)
panel_stat['codmpio']=panel_stat['codmpio'].astype(int)

panel_stat=panel_stat.set_geometry('centroids')
panel_stat=panel_stat.sjoin(grid_cc, how='left', predicate='intersects')
panel_stat['codmpio']=panel_stat['codmpio_left']
panel_stat.drop(columns=['dist_mun', 'centroids','codmpio_right', 'MPIO_CCDGO_right', 
                         'DPTO_CCDGO_right', 'id_right', 'index_right', 'codmpio_left',
                        'MPIO_CCDGO_left', 'DPTO_CCDGO_left'], inplace=True)
panel_stat=panel_stat.set_geometry('geometry')

len(panel_stat)

panel_stat=panel_stat.merge(violencia, how='left')

len(panel_stat)

panel_stat.to_parquet('/Users/santiagoherreragarcia/Library/CloudStorage/OneDrive-UniversidaddelosAndes/Semestre 2022-2/Econometria Avanzada/Trabajo Final/inputs/panel_stat.parquet')

len(grid_cc)

54718*20

panel_stat=gpd.read_parquet('/Users/santiagoherreragarcia/Library/CloudStorage/OneDrive-UniversidaddelosAndes/Semestre 2022-2/Econometria Avanzada/Trabajo Final/inputs/panel_stat.parquet')

panel_stat.head()

panel_stat.columns

2685700/20